# dosyayı okuma yapıyoruz


In [ ]:
import os.path
import joblib
raw_documents = []
snippets = []
with open("batı.csv" ,"r", encoding="utf8") as fin:
    for line in fin.readlines():
        text = line.strip()
        raw_documents.append( text )
        # her makale için başlık olarak 100 karaktere kadar kısa bir parça tutun
        snippets.append( text[0:min(len(text),100)] )
print("Read %d raw text documents" % len(raw_documents))


# stopword okuma

In [ ]:
custom_stop_words = []
with open( "stopwords.txt", "r", encoding = "utf-8") as fin:
    for line in fin.readlines():
        custom_stop_words.append( line.strip() )
print("Stopword list has %d entries" % len(custom_stop_words) )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#özel bir engellenecek kelimeler listesi ve minimum stopword sıklığını 20'ye ayarlayın
vectorizer = CountVectorizer(stop_words = custom_stop_words, min_df = 20)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d document-term matrix" % (A.shape[0], A.shape[1]) )


# kaç farklı kelime var bakıyoruz

In [ ]:
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

# Tek bir dosyada kalıcı hale getiriyoruz

In [ ]:
import joblib
joblib.dump((A,terms,snippets), "articles-raw.pkl") 

#### TF-IDF  Bir belge koleksiyonundaki bir kelimenin bir belgeyle ne kadar alakalı olduğunu değerlendiren istatistiksel bir ölçüdür. Bu, iki ölçümün çarpılmasıyla yapılır: bir belgede bir kelimenin kaç kez göründüğü ve bir dizi belgede kelimenin ters belge sıklığı.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#aynı ön işleme parametrelerini iletiyoruz
vectorizer = TfidfVectorizer(stop_words=custom_stop_words, min_df = 20)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]) )

In [ ]:
#ortaya çıkan kelime dağarcığını çıkarıyoruz
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

# Ağırlıklarını sıralıyoruz

In [ ]:
import operator
def rank_terms( A, terms ):
    #her sütundaki toplamları al
    sums = A.sum(axis=0)
    #ağırlıkları terimlere eşleyin
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # ağırlıklarına göre sıralıyoruz
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
ranking = rank_terms( A, terms )
for i, pair in enumerate( ranking[0:20] ):
    print( "%02d. %s (%.2f)" % ( i+1, pair[0], pair[1] ) )

In [ ]:
joblib.dump((A,terms,snippets), "articles-tfidf.pkl") 